In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import scipy
from scipy import sparse

In [2]:
df = pd.read_csv('../data/raw/ml-10M100K/ratings.dat', 
                sep='::',  usecols=[0,1,2,3], 
                names=['user', 'movie', 'rating', 'timestamp'])

df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
df.sort_values(by='timestamp', inplace=True)
df.drop(['timestamp'], axis=1, inplace=True)
df = df.pivot(index='user', columns='movie', values='rating').fillna(0)
data = sparse.coo_matrix(df)


c:\Users\ciril\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [3]:
df = pd.read_csv('../data/raw/ml-10M100K/ratings.dat', 
                sep='::',  usecols=[0,1,2,3], 
                names=['user', 'movie', 'rating', 'timestamp'])


In [72]:
df

,user,movie,rating,timestamp,is_valid
0,1,122,5.0,838985046,True
1,1,185,5.0,838983525,False
2,1,231,5.0,838983392,False
3,1,292,5.0,838983421,False
4,1,316,5.0,838983392,False
...,...,...,...,...,...
10000049,71567,2107,1.0,912580553,False
10000050,71567,2126,2.0,912649143,False
10000051,71567,2294,5.0,912577968,False
10000052,71567,2338,2.0,912578016,False


In [109]:
def get_last_n_ratings_by_user(df, n):
    new_df = df.groupby('user').filter(lambda x: len(x) >= n).sort_values('timestamp').groupby('user').tail(n).sort_values('user') #use rank()
    return new_df
    
def mark_last_n_ratings_as_validation_set(
    df, n):
    df["is_valid"] = False
    df.loc[
        get_last_n_ratings_by_user(df, n).index,
        "is_valid",
    ] = True
    return df
new_df = mark_last_n_ratings_as_validation_set(df, 5)

In [110]:
df.movie.nunique()

10677

In [111]:
from sklearn import preprocessing
le_users = preprocessing.LabelEncoder()
le_movies = preprocessing.LabelEncoder()
le_users = le_users.fit(new_df.user)
le_movies = le_movies.fit(new_df.movie)

In [112]:
train_df = new_df[new_df.is_valid==False]
train_df.user = le_users.transform(train_df.user)
train_df.movie = le_movies.transform(train_df.movie)
train_df = train_df.sort_values('user')

valid_df = new_df[new_df.is_valid==True]
valid_df.user = le_users.transform(valid_df.user)
valid_df.movie = le_movies.transform(valid_df.movie)
valid_df = valid_df.sort_values('user')

c:\Users\ciril\anaconda3\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [113]:
valid_df

,user,movie,rating,timestamp,is_valid
0,0,120,5.0,838985046,True
8,0,358,5.0,838984885,True
13,0,462,5.0,838984679,True
15,0,516,5.0,838984679,True
21,0,610,5.0,838984941,True
...,...,...,...,...,...
10000032,69877,1661,4.0,912649271,True
10000005,69877,257,5.0,912649271,True
10000014,69877,881,4.0,912649403,True
10000017,69877,1171,4.0,912649434,True


In [114]:
trs = sparse.coo_matrix((train_df['rating'], (train_df['user'], train_df['movie'])),
 shape=(train_df.user.max()+1, train_df.movie.max()+1)) # use label_encoder
trs

<69878x10677 sparse matrix of type '<class 'numpy.float64'>'
	with 9650664 stored elements in COOrdinate format>

In [115]:
tss = sparse.coo_matrix((valid_df['rating'], (valid_df['user'], valid_df['movie'])),
 shape=(valid_df.user.max()+1, valid_df.movie.max()+1)) # use label_encoder
tss

<69878x10677 sparse matrix of type '<class 'numpy.float64'>'
	with 349390 stored elements in COOrdinate format>

In [24]:
sparse.save_npz("../data/interim/train.npz", trs)
sparse.save_npz("../data/interim/test.npz", tss)

In [18]:
#train_df = train_df.pivot(index='user', columns='movie', values='rating').fillna(0)
#valid_df = valid_df.pivot(index='user', columns='movie', values='rating').fillna(0)

In [125]:
print(tss)

  (0, 120)	5.0
  (0, 358)	5.0
  (0, 462)	5.0
  (0, 516)	5.0
  (0, 610)	5.0
  (1, 149)	3.0
  (1, 535)	3.0
  (1, 707)	3.0
  (1, 1360)	3.0
  (1, 1494)	3.0
  (2, 9124)	3.5
  (2, 7041)	3.5
  (2, 3322)	4.0
  (2, 5203)	3.0
  (2, 5431)	4.5
  (3, 38)	3.0
  (3, 263)	5.0
  (3, 428)	3.0
  (3, 431)	3.0
  (3, 436)	3.0
  (4, 1271)	4.0
  (4, 1149)	4.0
  (4, 27)	3.0
  (4, 909)	5.0
  (4, 952)	4.0
  :	:
  (69873, 9458)	4.5
  (69873, 5889)	4.0
  (69873, 9117)	3.5
  (69873, 2805)	3.5
  (69873, 4290)	4.5
  (69874, 4228)	4.0
  (69874, 5322)	4.0
  (69874, 5845)	1.0
  (69874, 6118)	3.5
  (69874, 6427)	4.0
  (69875, 3171)	3.0
  (69875, 2544)	4.0
  (69875, 3074)	5.0
  (69875, 653)	4.0
  (69875, 2488)	5.0
  (69876, 110)	4.0
  (69876, 123)	4.0
  (69876, 133)	3.0
  (69876, 229)	4.0
  (69876, 575)	4.0
  (69877, 1661)	4.0
  (69877, 257)	5.0
  (69877, 881)	4.0
  (69877, 1171)	4.0
  (69877, 1752)	3.0


In [127]:
len(tss.tocsr().indices)

349390

In [128]:
users, items = np.repeat(range(tss.shape[0]), 5), tss.tocsr().indices

In [135]:
tss.tocsr().getnnz()

349390

In [119]:
from lightfm import LightFM
model = LightFM(loss='warp')
model.fit(trs, epochs=20, num_threads=4, verbose=True)

Epoch: 100%|██████████| 20/20 [06:31<00:00, 19.57s/it]


In [129]:
predictions = model.predict(users, items)
predictions

array([1.9077792, 2.831175 , 2.9762523, ..., 2.5008078, 1.3352544,
       0.5237628], dtype=float32)

In [131]:
from lightfm.evaluation import precision_at_k, recall_at_k

test_precision = precision_at_k(model, tss, k=5).mean()

In [132]:
test_precision

0.015770342

In [5]:
train_data = sparse.load_npz('../data/processed/train.npz')
test_data = sparse.load_npz('../data/processed/test.npz')

In [26]:
from lightfm import LightFM
#from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k, recall_at_k

# Load the MovieLens 100k dataset. Only five
# star ratings are treated as positive.
 # data = fetch_movielens(min_rating=1.0)
# Instantiate and train the model
model = LightFM(loss='warp')
model.fit(trs, epochs=20, num_threads=4)
model.fit(trs, epochs=20)

# Evaluate the trained model
test_precision = precision_at_k(model, tss, k=5).mean()

In [28]:
test_precision

0.3216692